# Intrinsic Valuation Functions

In [34]:
# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html
def country_def_spread(country):
    # AKA bond default spread
    #
    # Three ways
    # 1) government bonds 
    # def_spread = sov bond rate ('currency') - rfr('currency')
    # USD bond rate - USD riskfree rate = def spread
    # poland: .0468 - .0388 = .008
    # 2) find CDS for sovereign emerging government
    # US CDS = .0032
    # poland CDS = .0146
    # poland CDS net of US = .0113
    # 3) Lookup def spread based on rating
    if country=='USA':
        return 0.0
    if country=='Brazil':
        return 0.0368
    if country=='Germany':
        return 0.0
    if country=='Japan':
        return 0.0086
    if country=='Brazil Test':
        return .0368
    
# http://www.worldgovernmentbonds.com/country/
def govt_bond_rate(country):
    if country=='USA':
        return 0.03435
    if country=='Brazil':
        return 0.12604
    if country=='Germany':
        return 0.02481
    if country=='Japan':
        return 0.00461
    if country=='Brazil Test':
        return 0.1276
    
def risk_free_rate(country):
    if country=='USA Embraer Test':
        return 0.04 
    
    return govt_bond_rate(country) - country_def_spread(country)

def relative_eq_mkt_vol(country):
    if country=='Brazil Test':
        stdev_index = .3
        stdev_bonds = .2
    
    return stdev_index/stdev_bonds

# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html
# TODO put data and test cases into a separate spreadsheet
def country_risk_premium(country):
    
    if country=='Mature Embraer Test':
        return .0
    if country=='Brazil Embraer Test':
        return .0789
    if country=='Argentina Ambev Test':
        return .09
    if country=='Bolivia Ambev Test':
        return .0488
    if country=='Brazil Ambev Test':
        return .0263
    if country=='Canada Ambev Test':
        return .0
    if country=='Chile Ambev Test':
        return .0105
    if country=='Ecuador Ambev Test':
        return .0675
    if country=='Paraguay Ambev Test':
        return .06
    if country=='Peru Ambev Test':
        return .03
    if country=='Western Europe Coca Cola Test':
        return .0067
    if country=='Eastern Europe Russia Coca Cola Test':
        return .0260
    if country=='Asia Coca Cola Test':
        return .0163
    if country=='Latin America Coca Cola Test':
        return .0342
    if country=='Australia Coca Cola Test':
        return .0
    if country=='Africa Coca Cola Test':
        return .0382
    if country=='North America Coca Cola Test':
        return .0
    if country=='Denmark Shell Test':
        return .0620
    if country=='Italy Shell Test':
        return .0914
    if country=='Norway Shell Test':
        return .0620
    if country=='UK Shell Test':
        return .0681
    if country=='Europe Shell Test':
        return .0740
    if country=='Brunei Shell Test':
        return .0904
    if country=='Iraq Shell Test':
        return .1137
    if country=='Malaysia Shell Test':
        return .0805
    if country=='Oman Shell Test':
        return .0729
    if country=='Russia Shell Test':
        return .1006
    if country=='Asia Middle East Shell Test':
        return .0774
    if country=='Oceania Shell Test':
        return .0620
    if country=='Gabon Shell Test':
        return .1176
    if country=='Nigeria Shell Test':
        return .1176
    if country=='Africa Shell Test':
        return .1217
    if country=='USA Shell Test':
        return .0620
    if country=='Canada Shell Test':
        return .0620
    if country=='Brazil Shell Test':
        return .0960
    if country=='Latin America Shell Test':
        return .1078

    return country_def_spread(country)*relative_eq_mkt_vol(country)

def weighted_country_risk_premium(countries, weights):
    if len(countries)!=len(weights):
        raise Exception("Countries and weights must be same length")
    
    # Initialize CRP and add risk components by country
    wcrp=0
    for i,cf in enumerate(countries):
        wcrp += weights[i]*country_risk_premium(countries[i])
    return wcrp

def market_premium(mkt_country):
    if (mkt_country=='USA Embraer Test'):
        return .05    
    
    # TODO compute/get IERP
    if (mkt_country=='USA') or (mkt_country=='Mature'):
        return .0594
    
# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html
def country_equity_risk_premium(mkt_country,countries,weights):
    return market_premium(mkt_country) + weighted_country_risk_premium(countries,weights)

def cost_of_equity(crp_type,mkt_country,risk_countries,risk_weights,beta,lambda_risk):

    # country = either country of incorporation/trading OR 
    # list of 
    
    # Method 1: Assume every company in country is equally exposed to country risk
    if crp_type=="equal_country_risk":
        return risk_free_rate(mkt_country) + beta*market_premium(mkt_country) + weighted_country_risk_premium(risk_countries,risk_weights)

    # Method 2: Assume company risk in country is similar to mature market risk
    if crp_type=="constant_beta":
        return risk_free_rate(mkt_country) + beta*(market_premium(mkt_country) + weighted_country_risk_premium(risk_countries,risk_weights))

    # Method 3: Country risk is separate risk factor
    if crp_type=="lambda_factors":
        return risk_free_rate(mkt_country) + beta*market_premium(mkt_country) + lambda_risk*weighted_country_risk_premium(risk_countries,risk_weights)

def weights(raw_numbers):
    sum=0
    for number in raw_numbers:
        sum+=number
        
    return [number/sum for number in raw_numbers]

def unlevered_beta(business):
    if business=='Metals and Mining Vale Test': return 0.86 
    if business=='Iron Ore Vale Test': return 0.83 
    if business=='Fertilizers Vale Test': return 0.99 
    if business=='Logistics Vale Test': return 0.75
    if business=='Aerospace Test': return 0.95

def bottom_up_beta(business_mix, business_weights, tax_rate, debt_to_equity):
    unlev_beta=0.
    cumsum=0.
    for i,business in enumerate(business_mix):
        unlev_compo = unlevered_beta(business)*business_weights[i]
        cumsum+=business_weights[i]
        unlev_beta += unlev_compo
     
    unlev_beta /= cumsum # normalize #TODO - remove from bottom up calc and make a separate function
    print(unlev_beta)
    return unlev_beta*(1+(1-tax_rate)*(debt_to_equity))


# Slide 24
# def coe(rfr,beta,erp):
#     return rfr + beta*erp


# Long term cost to borrow?
def pre_tax_rate(country):
    if country=='test':
        return .1
        
def tax_rate(country):
    if country=='test': return .5
    if country=='Brazil Test': return .34
    
def cod(country):
    return pre_tax_rate(country)*(1 - tax_rate(country))
    
def wacc(mkt_equity, coe, mkt_debt, codebt):
    return coe*mkt_equity/(mkt_equity + mkt_debt) + codebt*mkt_debt/(mkt_equity + mkt_debt)
    
    
def dcf(rate,flow,term_val):
    if len(rate)!=len(flow):
        raise Exception("Cash flows and rates must be same length")
    
    # Initialize value, add cash flows, add terminal value
    value=0
    for i,cf in enumerate(flow):
        value += flow[i]/(1 + rate[i])**(i+1)
    value += term_val/(1 + rate[len(flow)-1])**(len(flow))
    
    return value

# Country and Currency Parameters

In [35]:
country = 'USA'
currency = 'USD'

# Company Data for Testing

In [36]:
# https://pages.stern.nyu.edu/~adamodar/pdfiles/eqnotes/valpacket1spr23.pdf
# Slides 8-10

# Equity test case - cost of equity, CF to equity, equity terminal value
test_coe = .13625
test_coeq = [test_coe, test_coe, test_coe, test_coe, test_coe]
test_eq_flow = [50,60,68,76.2,83.49]
test_eq_tv = 1603.0
test_eq_val = dcf(test_coeq,test_eq_flow,test_eq_tv)

# Firm case - cost of capital, CF to firm, firm terminal value
test_mkt_debt = 800
test_cod = cod('test')
test_coc = wacc(test_eq_val, test_coe, test_mkt_debt, test_cod)
test_cocap = [test_coc, test_coc, test_coc, test_coc, test_coc]
test_firm_flow = [90,100,108,116.2,123.49]
test_firm_tv = 2363.008
test_firm_val = dcf(test_cocap,test_firm_flow,test_firm_tv)

# Corporate risk premium calculations: slides 56-59
embraer_countries = ['Mature Embraer Test',
                     'Brazil Embraer Test']
embraer_weights = [.97, .03]

ambev_countries = ['Argentina Ambev Test',
                   'Bolivia Ambev Test',
                   'Brazil Ambev Test',
                   'Canada Ambev Test',
                   'Chile Ambev Test',
                   'Ecuador Ambev Test',
                   'Paraguay Ambev Test',
                   'Peru Ambev Test']
ambev_weights = [.0931, .0196, .6373, .1127, .0343, .0294, .0147, .0588]

cocacola_countries = ['Western Europe Coca Cola Test',
                      'Eastern Europe Russia Coca Cola Test',
                      'Asia Coca Cola Test',
                      'Latin America Coca Cola Test',
                      'Australia Coca Cola Test',
                      'Africa Coca Cola Test',
                      'North America Coca Cola Test']
cocacola_weights = [.19, .05, .15, .15, .04, .04, .4]

shell_countries = ['Denmark Shell Test',
                   'Italy Shell Test',
                   'Norway Shell Test',
                   'UK Shell Test',
                   'Europe Shell Test',
                   'Brunei Shell Test',
                   'Iraq Shell Test',
                   'Malaysia Shell Test',
                   'Oman Shell Test',
                   'Russia Shell Test',
                   'Asia Middle East Shell Test',
                   'Oceania Shell Test',
                   'Gabon Shell Test',
                   'Nigeria Shell Test',
                   'Africa Shell Test',
                   'USA Shell Test',
                   'Canada Shell Test',
                   'Brazil Shell Test',
                   'Latin America Shell Test']
shell_weights = [.0383, .0246, .0316, .0457, .0019, 
                 .0018, .0440, .0506, .1726, .0485, 
                 .0539, .0173, .0275, .1493, .0136,
                 .2295, .0189, .0293, .0013]

vale_bus_weights = weights([17739.0,81188.0,5741.0,1874.0])
vale_bus_sectors = ['Metals and Mining Vale Test', 
                    'Iron Ore Vale Test', 
                    'Fertilizers Vale Test', 
                    'Logistics Vale Test']

embraer_bus_weights = weights([1.])
embraer_bus_sectors = ['Aerospace Test']

# Value Calculation Tests
Yes I know these are not actual tests

In [37]:
print(test_eq_val)
print(test_firm_val)
print(risk_free_rate('Brazil Test')) # slide 38, should be .0908
print(country_equity_risk_premium('USA Embraer Test',['Brazil Test'],[1.0])) # slide 51, should be .1146
print()

print(weighted_country_risk_premium(embraer_countries,embraer_weights)) # slide 56, should be .0024
print(weighted_country_risk_premium(ambev_countries,ambev_weights)) # slide 56, should be .0311
print(weighted_country_risk_premium(cocacola_countries,cocacola_weights)) # slide 57, should be .0114 TODO
print(weighted_country_risk_premium(shell_countries,shell_weights)) # slide 59, should be .0826
print()

print(cost_of_equity("equal_country_risk","USA Embraer Test",['Brazil Embraer Test'],[1.0],1.07,0)) # Slide 63, should be .1724
print(cost_of_equity("equal_country_risk","USA Embraer Test",embraer_countries,embraer_weights,1.07,0)) # Slide 63, should be .0959
print(cost_of_equity("constant_beta","USA Embraer Test",['Brazil Embraer Test'],[1.0],1.07,0)) # Slide 63, should be .1779
print(cost_of_equity("constant_beta","USA Embraer Test",embraer_countries,embraer_weights,1.07,0)) # Slide 63, should be .0960
print(cost_of_equity("lambda_factors","USA Embraer Test",['Brazil Embraer Test'],[1.0],1.07,0.27)) # Slide 63, should be .1148
print()

print("Vale business weights from slide 92, should be 16.65, 76.20, 5.39, 1.76")
print(vale_bus_weights)
print()

print("Vale Bottum-up Beta from slide 92, should be 1.1503")
print(bottom_up_beta(vale_bus_sectors, vale_bus_weights, tax_rate('Brazil Test'), .5499))
print()

print("Embraer Bottum-up Beta from slide 93, should be 1.07")
print(bottom_up_beta(embraer_bus_sectors, embraer_bus_weights, tax_rate('Brazil Test'), .1895))
print()

1073.0065063585455
1873.4651511854781
0.09079999999999999
0.10519999999999999

0.0023669999999999997
0.031087120000000003
0.011676
0.08258047

0.1724
0.095867
0.17792300000000003
0.09603269
0.114803

Vale business weights from slide 92, should be 16.65, 76.20, 5.39, 1.76
[0.1664977192093259, 0.7620281203656774, 0.05388485292185242, 0.0175893075031443]

Vale Bottum-up Beta from slide 92, should be 1.1503
0.8422093634435247
1.147875776555537

Embraer Bottum-up Beta from slide 93, should be 1.07
0.95
1.0688165



In [27]:
tr = tax_rate('Brazil Test')
0.8422093634435247*(1+(1-tr)*(.55))

1.1479313623735241

# Valuation
Bell Textron is my first case

In [53]:
### INPUTS
bell_mkt_country = 'USA' # Publicly traded on US market
bell_inc_country = 'USA' # Incorporated in USA for tax purposes

bell_bus_sectors = ['Aircraft',
                    'Aftermarket parts and services',
                    'Military aircraft and support programs',
                    'Commercial helicopters, parts and services',
                    'Textron Systems',
                    'Fuel systems and functional components',
                    'Specialized vehicles',
                    'Textron eAviation',
                    'Finance']
bell_bus_rev = [718,
                 431,
                 385,
                 236,
                 306,
                 488,
                 444,
                 4,
                 12]
bell_bus_weights = weights(bell_bus_revs)
# TODO bell_debt_equity = 

bell_ops_countries = ['USA','Europe','International'] # From 10Q note on revenues
bell_ops_rev = [2070,
                305,
                649]
bell_ops_weights = weights(bell_ops_rev)

# Cost of Equity
# TODO bell_coe = risk_free_rate(bell_mkt_country)

In [54]:
print(bell_bus_weights)
print(sum(bell_bus_rev)) #3024
print(sum(bell_ops_rev)) #3024

[0.23743386243386244, 0.14252645502645503, 0.12731481481481483, 0.07804232804232804, 0.10119047619047619, 0.16137566137566137, 0.14682539682539683, 0.0013227513227513227, 0.003968253968253968]
3024
3024
